# Connecting to a Store and Adding Datasets

In this Notebook we create a new store and add a few datasets to it.

## Connect to store (using sina local file)

First let's create an empty databse (with you as a single user)

In a real application only admin user should have write permission to the file

In [1]:
import os
import sys
import shlex
from subprocess import Popen, PIPE
import kosh

kosh_example_sql_file = "kosh_example.sql"

# Create a new store (erase if exists)
store = kosh.connect(kosh_example_sql_file, delete_all_contents=True)

In [2]:
from  kosh import connect
import os

# connect to store
store = connect(kosh_example_sql_file)

## Adding datasets to the store

Let's add the first 10 runs

In [3]:
import glob
try:
    from tqdm.autonotebook import tqdm
except:
    tqdm = list

runs = glob.glob("sample_files/run*hdf5")
print("we found: {} runs".format(len(runs)))

for run in tqdm(runs[:10]):
    name = os.path.basename(run).split(".")[0]
    print("DATASET NAME:", name)
    # let's make sure it is unique, in case we run this cell multiple times
    datasets = list(store.find(name=name))
    if len(datasets) == 0:
        store.create(name)
    else:
        print("we found {} datasets already matching this name".format(len(datasets)))
        print(datasets[0])


we found: 125 runs


/g/g19/cdoutrix/miniconda3/envs/kosh/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/10 [00:00<?, ?it/s]

DATASET NAME: run_000
DATASET NAME: run_001
DATASET NAME: run_002
DATASET NAME: run_003
DATASET NAME: run_004
DATASET NAME: run_005
DATASET NAME: run_006
DATASET NAME: run_007
DATASET NAME: run_008
DATASET NAME: run_009


## Adding attributes do a dataset

For each of these runs let's add metadata

In [4]:
import random

def create_metadata():
    metadata = {"param1": random.random() * 2.,
                "param2": random.random() * 1.5,
                "param3": random.random() * 5,
                "param4": random.random() * 3,
                "param5": random.random() * 2.5,
                "param6": chr(random.randint(65, 91)),
               }
    metadata["project"] = "Kosh Tutorial"
    return metadata

pbar = tqdm(runs[:10])
for run in pbar:
    name = os.path.basename(run).split(".")[0]
    # Retrieve dataset via name
    dataset = list(store.search(name=name))[0]
    # Let's create a few random attributes
    metadata = create_metadata()
    for attribute in metadata:
        setattr(dataset, attribute, metadata[attribute])
print(dataset)

  0%|          | 0/10 [00:00<?, ?it/s]

KOSH DATASET
	id: f4e60cb29dac4dcbbbf51460af73b8a9
	name: run_009
	creator: cdoutrix

--- Attributes ---
	creator: cdoutrix
	name: run_009
	param1: 0.46033554077671224
	param2: 1.3555517113979287
	param3: 0.16588903647389464
	param4: 0.07910886294091912
	param5: 2.258409546589035
	param6: Z
	project: Kosh Tutorial
--- Associated Data (0)---
--- Ensembles (0)---
	[]


## Creating datasets with all the metadata at once.

Writing datasets attributes one at a time, meant accessing the store and editing every single time. This can be slow.

Let's speeds things up by writing all the attributes at once.

We will also turn to asynchronous mode on to speed up things further. This means we will only write to the store when the user says so. At that time Kosh will double check that nobody else changes any of these attributes while you were in async mode.

In [5]:
store.synchronous(False)
pbar = tqdm(runs[10:])
for i, run in enumerate(pbar):
    name = os.path.basename(run).split(".")[0]
    #pbar.set_description("run: {:45}".format(name))
    # let's make sure it is unique
    #datasets = store.search(name=name)
    datasets=[]
    if len(datasets) == 0:
        metadata = create_metadata()
        dataset = store.create(name, metadata=metadata)
    else:
        print("we found {} datasets already matching this name".format(len(datasets)))
        print(datasets[0])
print(dataset)
# We need to sync the store to ensure it's written to the database
store.sync()

  0%|          | 0/115 [00:00<?, ?it/s]

KOSH DATASET
	id: 339db962b00e4965950acd3d7eabc03f
	name: run_106
	creator: cdoutrix

--- Attributes ---
	creator: cdoutrix
	name: run_106
	param1: 0.1885545362201031
	param2: 1.3196681267696033
	param3: 1.9975781451268841
	param4: 2.0801721610187065
	param5: 1.3531907909084304
	param6: V
	project: Kosh Tutorial
--- Associated Data (0)---
--- Ensembles (0)---
	[]


## Adding/Modifying/Deleting Dataset attributes

In [6]:
# List existing attributes
print(dataset.listattributes())

['creator', 'id', 'name', 'param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'project']


In [7]:
# Create a new attribute
dataset.new_attribute = "new"
print(dataset.listattributes())
print(dataset.new_attribute)

['creator', 'id', 'name', 'new_attribute', 'param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'project']
new


In [8]:
# modify an attribute
dataset.new_attribute = "changed"
print(dataset.new_attribute)

changed


In [9]:
# Modify/add many attributes at once (less db access, faster)
dataset.update({"new_attribute": "changed_again", "yet_another_new_attribute":"yana"})
print(dataset.listattributes())
print(dataset.new_attribute)
print(dataset.yet_another_new_attribute)

['creator', 'id', 'name', 'new_attribute', 'param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'project', 'yet_another_new_attribute']
changed_again
yana


In [10]:
# Deleting attributes
del(dataset.new_attribute)
del(dataset.yet_another_new_attribute)
print(dataset.listattributes())

['creator', 'id', 'name', 'param1', 'param2', 'param3', 'param4', 'param5', 'param6', 'project']


## Deleting datasets from the store

In [11]:
store.delete(dataset)

## Querying the store

When querying the store use a dictionary to specify key/values we want to look for:

In [12]:
datasets = list(store.find(param6='B', ids_only=True))  # Only their ids (faster)
print(len(datasets))

4


Using sina's query capabilities we can use `ranges` [(more on sina utils here)](https://lc.llnl.gov/workflow/docs/sina/generated_docs/sina.utils.html)

In [13]:
from sina.utils import DataRange
datasets = list(store.find(param1=DataRange(min=1.7)))
print(len(datasets))

13


We can also search for datasets having a specific attribute (independently of its type or value)

In [14]:
datasets = list(store.find('param1'))
# or using sina's tools
from sina.utils import exists
datasets = list(store.find(param1=exists()))
print(len(datasets))

125
